In [85]:
import pandas as pd
import numpy as np
import openpyxl

In [86]:
table_source = np.zeros([10,3])
table_source[:,:] = np.nan
dataframe = pd.DataFrame(table_source,columns=['商品名', '価格', '個数'])
print(dataframe)

   商品名  価格  個数
0  NaN NaN NaN
1  NaN NaN NaN
2  NaN NaN NaN
3  NaN NaN NaN
4  NaN NaN NaN
5  NaN NaN NaN
6  NaN NaN NaN
7  NaN NaN NaN
8  NaN NaN NaN
9  NaN NaN NaN


In [87]:
menu_array = ['DXハンバーグ弁当', 'ハンバーグ弁当', '春巻き&ササミカツ弁当', 'ガパオライス', 'オムライス', 'サバカラ弁当', 'シャケカラ弁当', 'カラアゲ弁当']

quantity_array = [5,4,5,3,2,4,2,5]
for index, product_name in enumerate(menu_array):
    dataframe.loc[index, '商品名'] = product_name
for index, quantity in enumerate(quantity_array):
    dataframe.loc[index, '個数'] = quantity
dataframe

,商品名,価格,個数
0,DXハンバーグ弁当,NaN,5.0
1,ハンバーグ弁当,NaN,4.0
2,春巻き&ササミカツ弁当,NaN,5.0
3,ガパオライス,NaN,3.0
4,オムライス,NaN,2.0
5,サバカラ弁当,NaN,4.0
6,シャケカラ弁当,NaN,2.0
7,カラアゲ弁当,NaN,5.0
8,NaN,NaN,NaN
9,NaN,NaN,NaN


In [88]:
#エクセルファイル指定
workbook = openpyxl.load_workbook('calculator.xlsx')
#ワークシート指定
sheet = workbook['販売個数']
#最大行
maxRow = sheet.max_row + 1

col_name1 = 6
col_name2 = 7
for i in reversed(range(1, maxRow)):
    if sheet.cell(row=i, column=col_name1).value != None:
        for product_name, quantity in zip(dataframe['商品名'], dataframe['個数']):
            sheet.cell(i+1,col_name1,value = product_name)
            sheet.cell(i+1,col_name2,value = quantity)
            i = i + 1
        break

workbook.save('calculator.xlsx')